In [1]:
%pip install pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


## Dimensions et Fait

In [1]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, Float, Date, ForeignKey, ForeignKeyConstraint
from sqlalchemy.orm import Session, declarative_base, relationship

Base = declarative_base()

class DimDates(Base):
    __tablename__ = 'DimDates'
    dateID = Column(Integer, primary_key=True, autoincrement=True)
    time = Column(Date, unique=True)

class DimHumidity(Base):
    __tablename__ = 'DimHumidity'
    humidityID = Column(Integer, primary_key=True, autoincrement=True)
    relative_humidity_2m = Column(Float)

class DimCloudCovers(Base):
    __tablename__ = 'DimCloudCovers'
    cloudcoverID = Column(Integer, primary_key=True, autoincrement=True)
    cloud_cover = Column(Float)

class DimEvapotranspirations(Base):
    __tablename__ = 'DimEvapotranspirations'
    evapotranspirationID = Column(Integer, primary_key=True, autoincrement=True)
    et0_fao_evapotranspiration = Column(Float)

class DimTemperature(Base):
    __tablename__ = 'DimTemperature'
    temperatureID = Column(Integer, primary_key=True, autoincrement=True)
    temperature_2m = Column(Float)

class DimVPD(Base):
    __tablename__ = 'DimVPD'
    vpdID = Column(Integer, primary_key=True, autoincrement=True)
    vapour_pressure_deficit = Column(Float)

class DimRain(Base):
    __tablename__ = 'DimRain'
    rainID = Column(Integer, primary_key=True, autoincrement=True)
    rain = Column(Float)

class DimSnowfall(Base):
    __tablename__ = 'DimSnowfall'
    snowfallID = Column(Integer, primary_key=True, autoincrement=True)
    snowfall = Column(Float)
    snow_depth = Column(Float)

class DimWind(Base):
    __tablename__ = 'DimWind'
    windID = Column(Integer, primary_key=True, autoincrement=True)
    wind_speed_10m = Column(Float)
    wind_speed_100m = Column(Float)
    wind_direction_10m = Column(Float)
    wind_direction_100m = Column(Float)
    wind_gusts_10m = Column(Float)

import pandas as pd
from sqlalchemy import create_engine, Column, Integer, Float, Date, ForeignKey, ForeignKeyConstraint
from sqlalchemy.orm import Session, declarative_base, relationship

Base = declarative_base()

# Définir les classes de dimension comme précédemment...

class FactAirSoil(Base):
    __tablename__ = 'Fact_Air_Soil'
    fact_ID = Column(Integer, primary_key=True, autoincrement=True)
    time_id = Column(Integer, ForeignKey('DimDates.dateID'))
    cloud_cover_id = Column(Integer, ForeignKey('DimCloudCovers.cloudcoverID'))
    evapotranspiration_id = Column(Integer, ForeignKey('DimEvapotranspirations.evapotranspirationID'))
    temperature_id = Column(Integer, ForeignKey('DimTemperature.temperatureID'))
    vpd_id = Column(Integer, ForeignKey('DimVPD.vpdID'))
    rain_id = Column(Integer, ForeignKey('DimRain.rainID'))
    snowfall_id = Column(Integer, ForeignKey('DimSnowfall.snowfallID'))
    wind_id = Column(Integer, ForeignKey('DimWind.windID'))
    humidity_id = Column(Integer, ForeignKey('DimHumidity.humidityID'))

    # Ajoutez des mesures
    pm10 = Column(Float)
    pm2_5 = Column(Float)
    carbon_monoxide = Column(Float)
    nitrogen_dioxide = Column(Float)
    sulphur_dioxide = Column(Float)
    ozone = Column(Float)
    soil_temperature_0_to_7cm = Column(Float)
    soil_temperature_7_to_28cm = Column(Float)
    soil_temperature_28_to_100cm = Column(Float)
    soil_temperature_100_to_255cm = Column(Float)
    soil_moisture_0_to_7cm = Column(Float)
    soil_moisture_7_to_28cm = Column(Float)
    soil_moisture_28_to_100cm = Column(Float)
    soil_moisture_100_to_255cm = Column(Float)

    # Ajoutez les relations avec les tables de dimension
    time = relationship('DimDates', foreign_keys=[time_id])
    cloud_cover = relationship('DimCloudCovers', foreign_keys=[cloud_cover_id])
    evapotranspiration = relationship('DimEvapotranspirations', foreign_keys=[evapotranspiration_id])
    temperature = relationship('DimTemperature', foreign_keys=[temperature_id])
    vpd = relationship('DimVPD', foreign_keys=[vpd_id])
    rain = relationship('DimRain', foreign_keys=[rain_id])
    snowfall = relationship('DimSnowfall', foreign_keys=[snowfall_id])
    wind = relationship('DimWind', foreign_keys=[wind_id])
    humidity = relationship('DimHumidity', foreign_keys=[humidity_id])

    # Définissez les contraintes de clé étrangère ici
    __table_args__ = (
        ForeignKeyConstraint(['cloud_cover_id'], ['DimCloudCovers.id']),
        ForeignKeyConstraint(['time_id'], ['DimDates.id']),
        ForeignKeyConstraint(['evapotranspiration_id'], ['DimEvapotranspirations.id']),
        ForeignKeyConstraint(['temperature_id'], ['DimTemperature.id']),
        ForeignKeyConstraint(['vpd_id'], ['DimVPD.id']),
        ForeignKeyConstraint(['rain_id'], ['DimRain.id']),
        ForeignKeyConstraint(['snowfall_id'], ['DimSnowfall.id']),
        ForeignKeyConstraint(['wind_id'], ['DimWind.id']),
        ForeignKeyConstraint(['humidity_id'], ['DimHumidity.id']),
    )

# Charger les données transformées
df = pd.read_csv("/home/FIL_ROUGE/Historical/data/processed/weatherAirSoil.csv")

# Convertir la colonne "time" en type datetime 
df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d-%H-%M-%S', errors='coerce')

# Supprimer les doublons dans la colonne 'time'
df = df.drop_duplicates(subset=['time']).reset_index(drop=True)

# Créer la chaîne de connexion SQLAlchemy
server = '172.16.11.132'
database = 'weather_warehouse'
username = 'fidele'
password = 'admin'
conn_str = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

# Créer le moteur SQLAlchemy
engine = create_engine(conn_str, echo=False)

# Créer la session SQLAlchemy
session = Session(engine)

try:
    conn = engine.connect()
    print("Connexion réussie.")

    # Insérer les données dans les tables de dimension
    df_dates = df[['time']].drop_duplicates().reset_index(drop=True)
    df_dates.to_sql('DimDates', conn, if_exists='replace', index=True, index_label='dateID', method='multi', chunksize=500)

    df_humidity = df[['relative_humidity_2m (%)']].drop_duplicates().reset_index(drop=True)
    df_humidity.to_sql('DimHumidity', conn, if_exists='replace', index=True, index_label='humidityID', method='multi', chunksize=500)

    df_cloud_cover = df[['cloud_cover (%)']].drop_duplicates().reset_index(drop=True)
    df_cloud_cover.to_sql('DimCloudCovers', conn, if_exists='replace', index=True, index_label='cloudcoverID', method='multi', chunksize=500)

    df_evapotranspiration = df[['et0_fao_evapotranspiration (mm)']].drop_duplicates().reset_index(drop=True)
    df_evapotranspiration.to_sql('DimEvapotranspirations', conn, if_exists='replace', index=True, index_label='evapotranspirationID', method='multi', chunksize=500)

    df_temperature = df[['temperature_2m (°C)']].drop_duplicates().reset_index(drop=True)
    df_temperature.to_sql('DimTemperature', conn, if_exists='replace', index=True, index_label='temperatureID', method='multi', chunksize=500)

    df_vpd = df[['vapour_pressure_deficit (kPa)']].drop_duplicates().reset_index(drop=True)
    df_vpd.to_sql('DimVPD', conn, if_exists='replace', index=True, index_label='vpdID', method='multi', chunksize=500)

    df_rain = df[['rain (mm)']].drop_duplicates().reset_index(drop=True)
    df_rain.to_sql('DimRain', conn, if_exists='replace', index=True, index_label='rainID', method='multi', chunksize=500)

    df_snowfall = df[['snowfall (cm)', 'snow_depth (m)']].drop_duplicates().reset_index(drop=True)
    df_snowfall.to_sql('DimSnowfall', conn, if_exists='replace', index=True, index_label='snowfallID', method='multi', chunksize=500)

    df_wind = df[['wind_speed_10m (m/s)', 'wind_direction_10m (°)', 'wind_gusts_10m (m/s)']].drop_duplicates().reset_index(drop=True)
    df_wind.to_sql('DimWind', conn, if_exists='replace', index=True, index_label='windID', method='multi', chunksize=500)

    df_fact = df[['time', 'cloud_cover (%)', 'et0_fao_evapotranspiration (mm)', 'temperature_2m (°C)', 'vapour_pressure_deficit (kPa)',
                  'rain (mm)', 'snowfall (cm)', 'wind_speed_10m (m/s)', 'relative_humidity_2m (%)',
                  'pm10 (μg/m³)', 'pm2_5 (μg/m³)', 'carbon_monoxide (μg/m³)', 'nitrogen_dioxide (μg/m³)',
                  'sulphur_dioxide (μg/m³)', 'ozone (μg/m³)', 'soil_temperature_0_to_7cm (°C)',
                  'soil_temperature_7_to_28cm (°C)', 'soil_temperature_28_to_100cm (°C)',
                  'soil_temperature_100_to_255cm (°C)', 'soil_moisture_0_to_7cm (m³/m³)',
                  'soil_moisture_7_to_28cm (m³/m³)', 'soil_moisture_28_to_100cm (m³/m³)',
                  'soil_moisture_100_to_255cm (m³/m³)']]

    # Ajouter les clés étrangères en utilisant une jointure avec les tables de dimension
    df_fact = pd.merge(df_fact, df_dates, how='inner', left_on=['time'], right_on=['time'])
    df_fact = pd.merge(df_fact, df_cloud_cover, how='inner', left_on=['cloud_cover (%)'], right_on=['cloud_cover (%)'])
    df_fact = pd.merge(df_fact, df_evapotranspiration, how='inner', left_on=['et0_fao_evapotranspiration (mm)'], right_on=['et0_fao_evapotranspiration (mm)'])
    df_fact = pd.merge(df_fact, df_temperature, how='inner', left_on=['temperature_2m (°C)'], right_on=['temperature_2m (°C)'])
    df_fact = pd.merge(df_fact, df_vpd, how='inner', left_on=['vapour_pressure_deficit (kPa)'], right_on=['vapour_pressure_deficit (kPa)'])
    df_fact = pd.merge(df_fact, df_rain, how='inner', left_on=['rain (mm)'], right_on=['rain (mm)'])
    df_fact = pd.merge(df_fact, df_snowfall, how='inner', left_on=['snowfall (cm)'], right_on=['snowfall (cm)'])
    df_fact = pd.merge(df_fact, df_wind, how='inner', left_on=['wind_speed_10m (m/s)'], right_on=['wind_speed_10m (m/s)'])
    df_fact = pd.merge(df_fact, df_humidity, how='inner', left_on=['relative_humidity_2m (%)'], right_on=['relative_humidity_2m (%)'])

    # Supprimer les colonnes redondantes
    df_fact = df_fact[['time_id', 'cloud_cover_id', 'evapotranspiration_id', 'temperature_id', 'vpd_id', 'rain_id', 'snowfall_id', 'wind_id', 'humidity_id',
                       'pm10 (μg/m³)', 'pm2_5 (μg/m³)', 'carbon_monoxide (μg/m³)', 'nitrogen_dioxide (μg/m³)', 'sulphur_dioxide (μg/m³)',
                       'ozone (μg/m³)', 'soil_temperature_0_to_7cm (°C)', 'soil_temperature_7_to_28cm (°C)', 'soil_temperature_28_to_100cm (°C)',
                       'soil_temperature_100_to_255cm (°C)', 'soil_moisture_0_to_7cm (m³/m³)', 'soil_moisture_7_to_28cm (m³/m³)',
                       'soil_moisture_28_to_100cm (m³/m³)', 'soil_moisture_100_to_255cm (m³/m³)']]

    # Insérer les données dans la table de fait
    df_fact.to_sql('Fact_Air_Soil', conn, if_exists='replace', index=True, index_label='fact_ID', method='multi', chunksize=500)

    # Commit des changements
    session.commit()

except Exception as e:
    print(f"Erreur: {e}")

finally:
    if session:
        session.close()

Connexion réussie.


: 

In [20]:
df_dates.head()

,time
0,2022-07-29 06:00:00
1,2022-07-29 07:00:00
2,2022-07-29 08:00:00
3,2022-07-29 09:00:00
4,2022-07-29 10:00:00
